In [19]:
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier

In [20]:
df = pd.read_csv("../data/03_feature_engineered/2020-2021_engineered.csv")

In [21]:
df_2020_2021 = df.copy()

In [22]:
# drop columns about in-game stats 
df_2020_2021.drop(columns=['home_goals', 'away_goals', 'home_poss', 'away_poss', 'home_xg', 
                           'away_xg', 'home_sh', 'away_sh', 'home_shot_on_target', 'away_shot_on_target'], inplace=True)

In [23]:
df_2020_2021

,date,time,round,home_team,away_team,venue,result,home_form_last_5,home_avg_goals_last_5,home_avg_goals_conceded_last_5,home_avg_xg_last_5,home_avg_xg_conceded_last_5,home_avg_poss_last_5,home_avg_shots_last_5,home_avg_shots_on_target_last_5,away_form_last_5,away_avg_goals_last_5,away_avg_goals_conceded_last_5,away_avg_xg_last_5,away_avg_xg_conceded_last_5,away_avg_poss_last_5,away_avg_shots_last_5,away_avg_shots_on_target_last_5,home_Arsenal,home_Aston Villa,home_Brighton,home_Burnley,home_Chelsea,home_Crystal Palace,home_Everton,home_Fulham,home_Leeds United,home_Leicester City,home_Liverpool,home_Manchester City,home_Manchester Utd,home_Newcastle Utd,home_Sheffield Utd,home_Southampton,home_Tottenham,home_West Brom,home_West Ham,home_Wolves,away_Arsenal,away_Aston Villa,away_Brighton,away_Burnley,away_Chelsea,away_Crystal Palace,away_Everton,away_Fulham,away_Leeds United,away_Leicester City,away_Liverpool,away_Manchester City,away_Manchester Utd,away_Newcastle Utd,away_Sheffield Utd,away_Southampton,away_Tottenham,away_West Brom,away_West Ham,away_Wolves,form_difference,goals_difference,xg_difference,poss_difference,defensive_difference
0,2020-09-12,12:30,Matchweek 1,Fulham,Arsenal,Home,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,2020-09-12,15:00,Matchweek 1,Crystal Palace,Southampton,Home,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-09-12,17:30,Matchweek 1,Liverpool,Leeds United,Home,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,2020-09-12,20:00,Matchweek 1,West Ham,Newcastle Utd,Home,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
4,2020-09-13,14:00,Matchweek 1,West Brom,Leicester City,Home,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2021-05-23,16:00,Matchweek 38,Liverpool,Crystal Palace,Home,1,0.8,2.4,0.8,2.26,1.16,60.8,19.8,6.4,-0.2,1.4,2.0,1.46,1.36,39.4,15.0,5.8,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,0.80,21.4,1.2
376,2021-05-23,16:00,Matchweek 38,Manchester City,Everton,Home,1,0.2,2.2,1.8,1.42,1.12,61.4,14.2,3.8,0.0,0.6,0.6,1.12,1.00,44.8,14.0,4.2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.2,1.6,0.30,16.6,-1.2
377,2021-05-23,16:00,Matchweek 38,Sheffield Utd,Burnley,Home,1,-0.2,0.4,1.4,0.80,1.84,40.6,8.6,1.6,-0.2,1.4,1.8,1.52,1.56,41.6,12.2,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,-1.0,-0.72,-1.0,0.4
378,2021-05-23,16:00,Matchweek 38,West Ham,Southampton,Home,1,0.0,1.2,1.0,1.82,1.58,53.0,15.4,3.4,0.0,1.4,1.4,1.28,1.52,47.2,11.6,4.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,-0.2,0.54,5.8,0.4


In [17]:
# Get unique team names from the dataframe
team_names = df['home_team'].unique()

# Create a dictionary with team names as keys and 0 as values
team_points = {team: 0 for team in team_names}

In [ ]:
mw1 = df_2020_2021[df_2020_2021['round'] == 'Matchweek 1']

for row in mw1.itertuples():
    if row.result == 1:
        team_points[row.home_team] += 3
    elif row.result == -1:
        team_points[row.away_team] += 3
    else:
        # Corrected logic for a draw
        team_points[row.home_team] += 1
        team_points[row.away_team] += 1

{'Fulham': 0,
 'Crystal Palace': 0,
 'Liverpool': 0,
 'West Ham': 0,
 'West Brom': 0,
 'Tottenham': 0,
 'Sheffield Utd': 0,
 'Brighton': 0,
 'Everton': 0,
 'Leeds United': 0,
 'Manchester Utd': 0,
 'Arsenal': 0,
 'Southampton': 0,
 'Newcastle Utd': 0,
 'Chelsea': 0,
 'Leicester City': 0,
 'Aston Villa': 0,
 'Wolves': 0,
 'Burnley': 0,
 'Manchester City': 0}